# 파이썬 데이터베이스 연동(SQLite)
- https://sqlitestudio.pl/ 를 사용하여 db 생성 / 테이블 생성 및 수정, 삭제 이해하기
- 관계형 데이터베이스 관리 시스템을 제공하는 경량화된 소프트웨어 라이브러리
- 서버리스, 독립실행형, Zero-configuration, 트랜잭션 처리

- https://wikidocs.net/5327#google_vignette

## 1.DB 연결/ 테이블 생성 및 삽입

In [ ]:
import sqlite3

# 1. SQLite3 버전 출력
print('sqlite3.version:', sqlite3.version)
print('sqlite3.sqlite_version:', sqlite3.sqlite_version)

In [ ]:
try:
    # 2. DB 연결
    conn = sqlite3.connect('test.db')   # 데이터베이스와의 연결을 관리, SQL 실행 환경(커서)
    #conn = sqlite3.connect('test.db', isolation_level=None) # 자동 commit 옵션

    # 3. 커서 생성 : SQL 명령을 실행하고 결과를 가져오는 역할
    cur = conn.cursor()

    # 4. 테이블 생성
    cur.execute("""
        CREATE TABLE IF NOT EXISTS users(
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT,
            email TEXT,
            phone TEXT,
            website TEXT,
            regdate TEXT
        )
    """)
    print("Table 'users' checked/created successfully.")

    # 5. 데이터 삽입 (단일 레코드)
    cur.execute("INSERT INTO users VALUES (1, 'Kim', 'Kim@naver.com', '010-0000-0000', 'Kim.com', '2024-12-22 15:40:00')")
    cur.execute("INSERT INTO users(id, username, email, phone, website, regdate) VALUES (?, ?, ?, ?, ?, ?)",
                (2, 'Park', 'Park@naver.com', '010-1111-1111', 'Park.com', '2024-12-22 15:40:00'))

    # 6. 다중 데이터 삽입
    userList = [
        (3, 'Lee', 'Lee@naver.com', '010-2222-2222', 'Lee.com', '2024-12-22 15:40:00'),
        (4, 'Cho', 'Cho@naver.com', '010-3333-3333', 'Cho.com', '2024-12-22 15:40:00'),
        (5, 'Yoo', 'Yoo@naver.com', '010-4444-4444', 'Yoo.com', '2024-12-22 15:40:00')
    ]
    cur.executemany("INSERT INTO users(id, username, email, phone, website, regdate) VALUES (?, ?, ?, ?, ?, ?)", userList)
    print("Data inserted successfully.")

    # 7. 데이터 저장 (commit)
    conn.commit()
    print("Changes committed successfully.")

    # 8. 데이터 조회 (확인용)
    cur.execute("SELECT * FROM users")
    rows = cur.fetchall()
    print("Inserted Data:")
    for row in rows:
        print(row)

except sqlite3.Error as e:
    # 예외 처리
    print("Database error occurred:", e)

finally:
    # 9. 연결 종료
    if conn:
        conn.close()
        print("Database connection closed.")


## 2.DB 검색

In [ ]:
import sqlite3

# 1. DB 연결
conn = sqlite3.connect('test.db')  # 본인 DB 파일 경로

# 2. 커서 바인딩
cur = conn.cursor()

# 3. 데이터 조회(전체)
cur.execute("SELECT * FROM users")

# 3-1. 1개 로우 선택
print('One -> \n', cur.fetchone())

# 3-2. 지정된 개수의 로우 선택
print('Three -> \n', cur.fetchmany(size=3))

# 3-3. 전체 로우 선택
print('All -> \n', cur.fetchall())
print()

# 4. 순회 방식별 데이터 조회
# 4-1. `fetchall` 이후 순회 (조회 결과 없음, 커서가 이미 끝까지 이동했기 때문)
rows = cur.fetchall()
for row in rows:
    print('retrieve1  >', row)  # 조회 없음

# 4-2. 즉시 `fetchall`로 순회 (조회 결과 없음, 커서가 이미 끝까지 이동했기 때문)
for row in cur.fetchall():
    print('retrieve2 >', row)  # 조회 없음

# 4-3. SQL 재실행 후 순회
for row in cur.execute("SELECT * FROM users ORDER BY id DESC"):
    print('retrieve3 > ', row)
print()

# 5. 조건 검색 (WHERE 조건)
# 5-1. WHERE 절: 파라미터 전달 방식 (튜플 사용)
param1 = (1,)
cur.execute('SELECT * FROM users WHERE id=?', param1)
print('param1', cur.fetchone())  # 첫 번째 결과
print('param1', cur.fetchall())  # 나머지 결과

# 5-2. WHERE 절: 문자열 포맷팅 방식
param2 = 1
cur.execute("SELECT * FROM users WHERE id='%s'" % param2)  # %s 사용
print('param2', cur.fetchone())  # 첫 번째 결과
print('param2', cur.fetchall())  # 나머지 결과

# 5-3. WHERE 절: 딕셔너리를 사용한 파라미터 전달
cur.execute("SELECT * FROM users WHERE id= :Id", {"Id": 1})
print('param3', cur.fetchone())  # 첫 번째 결과
print('param3', cur.fetchall())  # 나머지 결과

# 5-4. WHERE 절: IN 조건 (튜플 사용)
param4 = (1, 4)
cur.execute('SELECT * FROM users WHERE id IN(?,?)', param4)
print('param4', cur.fetchall())

# 5-5. WHERE 절: IN 조건 (문자열 포맷팅 방식)
cur.execute("SELECT * FROM users WHERE id In('%d','%d')" % (1, 4))
print('param5', cur.fetchall())

# 5-6. WHERE 절: OR 조건 (딕셔너리 사용)
cur.execute("SELECT * FROM users WHERE id= :id1 OR id= :id2", {"id1": 1, "id2": 4})
print('param6', cur.fetchall())

# 6. 데이터베이스 덤프 출력 (백업 파일 생성)
with conn:
    with open('dump.sql', 'w') as f:
        for line in conn.iterdump():  # 데이터베이스 전체 스키마 및 데이터 덤프
            f.write('%s\n' % line)
        print('Dump Print Complete.')


## 3.테이블 수정 및 삭제

In [ ]:
import sqlite3

try:
    # 1. DB 연결
    conn = sqlite3.connect('test.db')  # SQLite 파일 연결
    print("Database connected successfully.")

    # 2. 커서 생성
    cur = conn.cursor()
    print("Cursor created successfully.")

    # 3. 데이터 수정
    # 3-1. 데이터 수정 방법 1: 튜플로 전달
    cur.execute("UPDATE users SET username = ? WHERE id = ?", ('niceman', 1))

    # 3-2. 데이터 수정 방법 2: 딕셔너리로 전달
    cur.execute("UPDATE users SET username = :name WHERE id = :id", {"name": 'niceman', "id": 3})

    # 3-3. 데이터 수정 방법 3: 문자열 포맷팅 사용
    cur.execute("UPDATE users SET username = '%s' WHERE id = '%s'" % ('badboy', 5))

    print("Data updated successfully.")

    # 4. 중간 데이터 확인 1
    print("Intermediate Data Check 1:")
    for user in cur.execute('SELECT * FROM users'):
        print(user)

    # 5. 데이터 삭제
    # 5-1. 데이터 삭제 방법 1: 튜플로 전달
    cur.execute("DELETE FROM users WHERE id = ?", (7,))

    # 5-2. 데이터 삭제 방법 2: 딕셔너리로 전달
    cur.execute("DELETE FROM users WHERE id = :id", {"id": 8})

    # 5-3. 데이터 삭제 방법 3: 문자열 포맷팅 사용
    cur.execute("DELETE FROM users WHERE id = '%s'" % 9)

    print("Rows deleted successfully.")

    # 6. 중간 데이터 확인 2
    print("Intermediate Data Check 2:")
    for user in cur.execute('SELECT * FROM users'):
        print(user)

    # 7. 테이블 전체 데이터 삭제
    deleted_rows = conn.execute("DELETE FROM users").rowcount
    print(f"All users deleted: {deleted_rows} rows affected.")

    # 8. 변경 사항 저장
    conn.commit()
    print("Changes committed successfully.")

except sqlite3.Error as e:
    # 예외 처리: 데이터베이스 오류 발생 시
    print("Database error occurred:", e)

finally:
    # 9. 연결 종료
    if conn:
        conn.close()
        print("Database connection closed.")


## 미션

In [ ]:
############################################################################################################
# 미션 : SQLite를 활용한 데이터베이스 생성 및 데이터 조작
# - Python 코드로 SQLite 데이터베이스를 생성하고 테이블과 데이터를 추가하세요.
# - 다음 요구사항을 만족하는 코드를 작성하세요:

# 요구사항:
# 1. 데이터베이스 파일 생성:
#    - 데이터베이스 파일 이름: `store.db`
#    - 해당 경로가 없으면 필요한 디렉토리를 자동으로 생성합니다.

# 2. 테이블 생성:
#    - 테이블 이름: `products`
#    - 테이블 스키마:
#      - id: INTEGER, PRIMARY KEY, AUTOINCREMENT (자동으로 증가하는 고유 식별자)
#      - name: TEXT, NOT NULL (제품 이름)
#      - price: REAL, NOT NULL (제품 가격)
#      - stock: INTEGER, NOT NULL (재고 수량)

# 3. 데이터 삽입:
#    - 다음 데이터를 삽입:
#      - 제품명: "Laptop", 가격: 1500.50, 재고: 10
#      - 제품명: "Mouse", 가격: 25.99, 재고: 100
#      - 제품명: "Keyboard", 가격: 45.00, 재고: 50

# 4. 데이터 조회:
#    - 테이블의 모든 데이터를 조회하여 출력합니다.

# 5. 예외 처리:
#    - 데이터베이스 작업 중 오류가 발생하면 트랜잭션을 롤백하여 데이터 손상을 방지합니다.

# 결과:
# - Python 코드를 실행하여 데이터베이스와 테이블이 생성됩니다.
# - 데이터베이스에 제품 정보가 삽입됩니다.
# - 삽입된 데이터가 콘솔에 출력됩니다.
# - 예외 발생 시 오류 메시지가 출력되고 데이터베이스 변경 사항이 롤백됩니다.

# 조건:
# - Python의 `sqlite3` 모듈을 사용합니다.
# - 코드 실행 시, 디렉토리 및 데이터베이스 파일이 없으면 자동 생성됩니다.
# - 테이블 이름과 데이터 스키마는 요구사항에 따라 작성해야 합니다.

# 주의:
# - 데이터베이스 파일(`store.db`)과 경로를 적절히 설정하세요.
# - 데이터 삽입 및 조회가 정확히 수행되도록 SQL 문법을 확인하세요.
# - 커서와 데이터베이스 연결을 적절히 종료하여 리소스 누수를 방지하세요.
############################################################################################################

In [2]:
import sqlite3
import os

# 1. 데이터베이스 파일 경로 설정
base_dir = os.getcwd()  # 현재 작업 디렉토리
db_locale = os.path.join(base_dir, 'store.db')

# 경로가 존재하지 않으면 디렉토리 생성
db_dir = os.path.dirname(db_locale)
if not os.path.exists(db_dir):
    os.makedirs(db_dir)

# 2. 데이터베이스 연결
conn = sqlite3.connect(db_locale)  # store.db 데이터베이스에 연결 (없으면 생성)

# 3. 커서 생성
cur = conn.cursor()  # SQL 작업을 수행할 커서를 생성

try:
    # 4. 테이블 생성
    cur.execute('''
        CREATE TABLE IF NOT EXISTS products (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            price REAL NOT NULL,
            stock INTEGER NOT NULL
        )
    ''')

    # 5. 데이터 삽입
    cur.execute("INSERT INTO products (name, price, stock) VALUES (?, ?, ?)", ("Laptop", 1500.50, 10))
    cur.execute("INSERT INTO products (name, price, stock) VALUES (?, ?, ?)", ("Mouse", 25.99, 100))
    cur.execute("INSERT INTO products (name, price, stock) VALUES (?, ?, ?)", ("Keyboard", 45.00, 50))

    # 6. 데이터 커밋
    conn.commit()

    # 7. 데이터 조회
    cur.execute("SELECT * FROM products")
    rows = cur.fetchall()  # 데이터 가져오기
    for row in rows:
        print(row)

# 8. 예외 발생 시 롤백
except Exception as e:
    conn.rollback()  # 트랜잭션 롤백
    print("Error:", e)

# 9. 종료 처리
finally:
    cur.close()  # 커서 닫기
    conn.close()  # 데이터베이스 연결 닫기

(1, 'Laptop', 1500.5, 10)
(2, 'Mouse', 25.99, 100)
(3, 'Keyboard', 45.0, 50)
